In [1]:
# %matplotlib inline
# import matplotlib.pyplot as plt

import io

import numpy as np
import cv2


from PIL import Image
import subprocess
import ultralytics
from ultralytics import YOLO
from ultralytics.utils import LOGGER
import logging

LOGGER.setLevel(logging.ERROR)

WARNING ⚠️ user config directory '/home/ubuntu/.config/Ultralytics' is not writeable, defaulting to '/tmp' or CWD. Alternatively you can define a YOLO_CONFIG_DIR environment variable for this path.
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/tmp/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
def get_b_array(image_path):
    with open(image_path, 'rb') as image_file:
        img1 = Image.open(image_file)
        
        # Convert the image to a byte stream in JPEG format
        img_byte_arr = io.BytesIO()
        img1.save(img_byte_arr, format='JPEG')  # or 'PNG' if you prefer
        img_byte_arr = img_byte_arr.getvalue()
    return img_byte_arr

In [3]:
def find_webcam_index(device_name):
    command = "v4l2-ctl --list-devices"
    output = subprocess.check_output(command, shell=True, text=True)
    devices = output.split('\n\n')
    
    for device in devices:
        if device_name in device:
            lines = device.split('\n')
            for line in lines:
                if "video" in line:
                    parts = line.split()
                    for part in parts:
                        if part.startswith('/dev/video'):
                            print(part)
                            return (part[10:])

In [4]:
webcam_index = int(find_webcam_index("C922")) #C922 #3D USB
webcam_index


/dev/video2


2

In [5]:
import ultralytics
from ultralytics import YOLO


In [6]:
model = YOLO("yolov8s.pt") 

In [7]:
width = 640 #320  # Desired width
height = 480 # 240  # Desired height
mult = 1
cap = cv2.VideoCapture(webcam_index)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

if not cap.isOpened():
    print("Error: Could not open video stream.")
    exit()

while True:
    # Read a frame from the camera
    ret, frame = cap.read()

    # If the frame was read correctly
    if not ret:
        print("Error: Failed to read frame.")
        break

    results = model(frame)[0]
    frame = results.plot()
    # Display the frame
    cv2.imshow('Camera Stream', frame)

    # Press 'q' to quit the stream
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.imwrite('frame.jpg', frame)
        break


# Release the capture and closeq the window
cap.release()
cv2.destroyAllWindows()